In [3]:
import sys; print(sys.prefix)

c:\users\sativa\medved\src-git\py\exp-jy-ntb\env


In [5]:
import requests
from bs4 import BeautifulSoup

In [33]:
H_USERAGENT={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
#r = requests.get('http://pyclass.com/example.html', headers=H_USERAGENT)
URL_EXAMPLE='http://pyclass.com/example.html'
#cont = r.content

In [51]:
def req_to_soup(url, hdrs = H_USERAGENT):
    r = requests.get(url, headers = hdrs)
    cont = r.content
    return BeautifulSoup(cont, "html.parser")
soup = req_to_soup(URL_EXAMPLE)
#print(soup.prettify())
divs = soup.find_all('div', {'class':'cities'})

In [27]:
print(divs)
print(len(divs))
print(type(divs))

[<div class="cities">
<h2>London</h2>
<p>London is the capital of England and it's been a British settlement since 2000 years ago. </p>
</div>, <div class="cities">
<h2>Paris</h2>
<p>Paris is the capital city of France. It was declared capital since 508.</p>
</div>, <div class="cities">
<h2>Tokyo</h2>
<p>Tokyo is the capital of Japan and one of the most populated cities in the world.</p>
</div>]
3
<class 'bs4.element.ResultSet'>


In [31]:
#print(divs[0].find_all("h2")[0].text) # single

for d in divs:
    print(d.find('h2').text)

London
London
Paris
Tokyo


In [52]:
# {'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
SRC_URL = "http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/"
html_estate = req_to_soup(SRC_URL)

In [63]:
rows = html_estate.find_all('div', {'class':'propertyRow'})

In [112]:
def normalize_price(text):
    return text.replace('\n', '').replace(' ', '')

def one_by_tag_class(html, tag, cname):
    return html.find(tag, {'class': cname})

def all_by_tag_class(html, tag, cname):
    return html.find_all(tag, {'class': cname})

def fetch_beds(item):
    try:
    #    print(item.find('span',{'class':'infoBed'}).find('b').text)
        result = one_by_tag_class(item, 'span', 'infoBed').find('b').text
    except: # XXX
        result = None
    return result

def items_from_url(url):
    items = []
    page = req_to_soup(url)
    for item in page.find_all('div', {'class':'propertyRow'}):
        d = {}
        raw_price = item.find('h4', {'class':'propPrice'}).text
        raw_price = one_by_tag_class(item, 'h4', 'propPrice').text
        price = normalize_price(raw_price)
        print(price)
        d['price'] = price

        beds = fetch_beds(item)
        print('Beds:', beds)
        d['beds'] = beds

        TAG_LOT_SIZE='Lot Size'
        for col_group in all_by_tag_class(item, 'div', 'columnGroup'):
            #print(col_group)
            for feat_group, feat_name in zip(all_by_tag_class(col_group, 'span', 'featureGroup'), all_by_tag_class(col_group, 'span', 'featureName')):
                #print(feat_group.text, feat_name.text) # everyting
                if TAG_LOT_SIZE in feat_group.text:
                    print('Lot size:', feat_name.text)
                    d['lsize'] = feat_name.text

        items.append(d)
        print('')
    return items

# ??
#page_nr=all_by_tag(soup, 'a', 'Page')[-1].text

items_list = []
for offset in range(0,30,10): # TODO: use page_nr
    url = SRC_URL + 't=0&s=' + str(offset) + '.html' # + '&subView=searchView.Paginate'
    print(url)
    items_list.extend( items_from_url(url) )

http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=0.html
$725,000
Beds: None

$452,900
Beds: 4
Lot size: 0.21 Acres

$396,900
Beds: 5

$389,900
Beds: 4
Lot size: Under 1/2 Acre, 

$254,000
Beds: 3
Lot size: Under 1/2 Acre, 

$252,900
Beds: 4
Lot size: 0.27 Acres

$210,000
Beds: 4
Lot size: Under 1/2 Acre, 

$209,000
Beds: 3
Lot size: Under 1/2 Acre, 

$199,900
Beds: 3
Lot size: Under 1/2 Acre, 

$196,900
Beds: 3
Lot size: 2.02 Acres

http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=10.html
$1,700,000
Beds: None

$1,100,000
Beds: None

$1,080,000
Beds: None

$908,350
Beds: None

$905,000
Beds: None

$695,000
Beds: None

$605,000
Beds: None

$559,805
Beds: None

$504,000
Beds: None

$410,000
Beds: None

http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=20.html
$299,900
Beds: 3

$167,500
Beds: 3

$150,000
Beds: None
Lot size: 5 Acres

$129,500
Beds: 3

$129,000
Beds: 4

$80,000
Beds: 5

$77,900
Beds: 3

$70,500
Beds: 3


In [111]:
import pandas
df = pandas.DataFrame(items_list)
df.to_csv('estate.csv')
df

,price,beds,lsize
0,"$725,000",None,NaN
1,"$452,900",4,0.21 Acres
2,"$396,900",5,NaN
3,"$389,900",4,"Under 1/2 Acre,"
4,"$254,000",3,"Under 1/2 Acre,"
5,"$252,900",4,0.27 Acres
6,"$210,000",4,"Under 1/2 Acre,"
7,"$209,000",3,"Under 1/2 Acre,"
8,"$199,900",3,"Under 1/2 Acre,"
9,"$196,900",3,2.02 Acres


In [106]:
df

""


In [89]:
tmp = []
tmp.extend([1,4])

In [90]:
tmp

[1, 4]